# Scenario Report

In [ ]:
# Copyright (c) 2022 The MATCH Authors. All rights reserved.
# Licensed under the GNU AFFERO GENERAL PUBLIC LICENSE Version 3 (or later), which is in the LICENSE file.

from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import numpy as np
from match_model.reporting.report_functions import *

#get the name of the current directory to specify the scenario name and identify the output directory
scenario_name = str(Path.cwd()).split('\\')[-1]
if scenario_name == 'inputs':
    data_dir = Path.cwd() / '../outputs/'
    inputs_dir = Path.cwd() / '../inputs/'
    scenario_output_dir = Path.cwd() / '../summary_reports/'
    scenario_name = 'N/A'
else:
    data_dir = Path.cwd()/ f'../../outputs/{scenario_name}/'
    inputs_dir = Path.cwd() / f'../../inputs/{scenario_name}/'
    scenario_output_dir = Path.cwd() / '../../summary_reports/'

#define formatting options/functions for outputs
pd.options.display.float_format = '{:,.2f}'.format

#allow the notebook to display plots in html report
###################################################
plotly.offline.init_notebook_mode()

print(f'Scenario Name: {scenario_name}')

In [ ]:
# load data from csvs
baseload_capacity_factors = pd.read_csv(inputs_dir / "baseload_capacity_factors.csv")
with open(inputs_dir / "cambium_region.txt", "r") as file:
    cambium_region = file.read()
with open(inputs_dir / "cambium_scenario.txt", "r") as file:
    cambium_scenario = file.read()
costs_by_gen = pd.read_csv(data_dir / "costs_by_gen.csv")
costs_by_tp = pd.read_csv(data_dir / "costs_by_tp.csv")
dispatch = pd.read_csv(data_dir / "dispatch.csv")
with open(inputs_dir / "ghg_emissions_unit.txt", "r") as unit:
    emissions_unit = unit.read()
financials = pd.read_csv(inputs_dir / "financials.csv")
fixed_costs = pd.read_csv(inputs_dir / "fixed_costs.csv")
gen_build_predetermined = pd.read_csv(inputs_dir / "gen_build_predetermined.csv")
gen_cap = pd.read_csv(data_dir / "gen_cap.csv")
generation_projects_info = pd.read_csv(inputs_dir / "generation_projects_info.csv")
with open(inputs_dir / "gen_set.txt", "r") as set_name:
    gen_set = set_name.read()
load_balance = pd.read_csv(data_dir / "load_balance.csv")
lrmer_data = pd.read_csv(inputs_dir / "lrmer_for_summary.csv")
nodal_prices = pd.read_csv(inputs_dir / "nodal_prices.csv")
periods = pd.read_csv(inputs_dir / "periods.csv")
# load RA data if modeled
try:
    ra_summary = pd.read_csv(data_dir / "RA_summary.csv")
    ra_exists = True
except FileNotFoundError:
    ra_exists = False
    ra_summary = pd.DataFrame()
rec_value = pd.read_csv(inputs_dir / "rec_value.csv")
system_power = pd.read_csv(data_dir / "system_power.csv")
with open(inputs_dir / "td_losses.txt", "r") as loss:
    td_losses = float(loss.read())
timestamps = pd.read_csv(
    inputs_dir / "timepoints.csv",
    parse_dates=["timestamp"],
    usecols=["timepoint_id", "timestamp"],
)
variable_capacity_factors = pd.read_csv(inputs_dir / "variable_capacity_factors.csv")
try:
    storage_builds = pd.read_csv(data_dir / "storage_builds.csv")
    storage_cycle_count = pd.read_csv(data_dir / "storage_cycle_count.csv")
    storage_dispatch = pd.read_csv(data_dir / "storage_dispatch.csv")
    # there may not be any hybrid storage
    try:
        hybrid_pair = hybrid_pair_dict(generation_projects_info)
    except KeyError:
        hybrid_pair = dict()
    storage_exists = True
except:
    storage_exists = False
    storage_builds = pd.DataFrame()
    storage_cycle_count = pd.DataFrame()
    storage_dispatch = pd.DataFrame()
    hybrid_pair = dict()

model_year = periods.loc[0, "period_start"]
financial_year = financials.loc[0, "dollar_year"]
base_year = financials.loc[0, "base_financial_year"]

technology_color_map = {
    "Small Hydro": "Blue",
    "Consumed Small Hydro": "Blue",
    "Excess Small Hydro": "DodgerBlue",
    "Onshore Wind": "DeepSkyBlue",
    "Consumed Onshore Wind": "DeepSkyBlue",
    "Excess Onshore Wind": "LightSkyBlue",
    "Offshore Wind": "Navy",
    "Consumed Offshore Wind": "Navy",
    "Excess Offshore Wind": "MediumSlateBlue",
    "Solar PV": "Gold",
    "Hybrid Solar PV": "Gold",
    "Consumed Solar PV": "Gold",
    "Excess Solar PV": "Yellow",
    "CSP": "Orange",
    "Geothermal": "Sienna",
    "Consumed Geothermal": "Sienna",
    "Storage": "Green",
    "Storage Discharge": "Green",
    "Hybrid Storage": "Green",
    "Grid Energy": "Red",
    "Shaped": "Orange",
    "Consumed Shaped": "Orange",
    "Excess Shaped": "LightSalmon",
    "Wave": "LightSeaGreen",
    "Solar Thermal": "Purple",
    "Consumed Solar Thermal": "Purple",
    "Excess Solar Thermal": "Plum",
    "(?)": "Black",
}



# Renewable Energy Goal

## Portfolio Renewable Percentage

In [ ]:
print(
    f"Time-coincident renewable percentage: {format_percent(hourly_renewable_percentage(load_balance))}"
)
print(
    f"Annual volumetric renewable percentage: {format_percent(annual_renewable_percentage(load_balance))}"
)
print(
    f"    Total Load:              {round(load_balance.zone_demand_mw.sum(), 1):,} MWh"
)
print(
    f"    Total Storage Losses:    {round(load_balance.ZoneTotalStorageCharge.sum() - load_balance.ZoneTotalStorageDischarge.sum(), 1):,} MWh"
)
print(
    f"    Total Excess Generation: {round(load_balance.ZoneTotalExcessGen.sum(), 1):,} MWh"
)


## Sensitivity Analysis
The following shows how well the selected portfolio would perform in specific resource years, considering the full intermittency and variability of wind and solar resources. 

In [ ]:
sensitivity_table = run_sensitivity_analysis(
    gen_set,
    gen_cap,
    dispatch,
    generation_projects_info,
    load_balance,
    storage_builds,
    storage_exists,
)



## Carbon footprint of delivered energy

In [ ]:
cambium_model_year = load_cambium_data(
    scenario=cambium_scenario, year=model_year, region=cambium_region
)
total_emissions = calculate_emissions(
    dispatch,
    generation_projects_info,
    system_power,
    load_balance,
    cambium_model_year,
    emissions_unit,
)
print(
    f"Using {model_year} residual-mix emissions factors calculated from the Cambium {cambium_scenario} scenario:"
)
print(
    f'Total Annual Emissions: {total_emissions["Total Emission Rate"].sum().round(1):,} {emissions_unit.split("/")[0]}'
)
print(
    f'Delivered Emission Factor: {round(total_emissions["Delivered Emission Factor"].mean(), 3)} {emissions_unit}'
)



In [ ]:
build_hourly_emissions_heatmap(total_emissions, emissions_unit, cambium_scenario).show()


# Generator Portfolio

The sunburst chart describes the built portfolio at various levels of detail, which shows how the outer rings relate to the inner rings
- Inner circle: contract status (contracted or additional project)
- Middle ring: technology type (e.g. solar, wind, ...)
- Outer ring: specific project name

For example, individual projects in the outer ring belong to a specific technology type in the middle ring, which can either be part of the existing/contracted portfolio, or the additional portfolio.

In [ ]:
portfolio = generator_portfolio(
    gen_cap, gen_build_predetermined, generation_projects_info, base_year
)

portfolio_sunburst = px.sunburst(
    portfolio,
    path=["Contract Status", "Build Status", "Technology", "generation_project"],
    values="MW",
    color="Technology",
    color_discrete_map=technology_color_map,
    width=1000,
    height=1000,
    title="Energy Portfolio by Project Name, Technology Type, Build Status, and Contract Status (MW)",
)
portfolio_sunburst.update_traces(textinfo="label+value")
portfolio_sunburst.show()


## Power Content Label


In [ ]:
power_content = power_content_label(load_balance, dispatch, generation_projects_info)

power_content["color"] = power_content["Source"].map(technology_color_map)

fig = make_subplots(
    rows=1,
    cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=["Time-coincident accounting", "Annual accounting"],
)
fig.add_trace(
    go.Pie(
        labels=power_content["Source"],
        values=power_content["Dispatched_MWh"],
        name="Time-coincident",
        sort=False,
    ),
    1,
    1,
)
fig.add_trace(
    go.Pie(
        labels=power_content["Source"],
        values=power_content["Total_MWh"],
        name="Annual",
        sort=False,
    ),
    1,
    2,
)

fig.update_layout(
    width=1200, height=600, title_text="Power Content of Delivered Energy"
)
fig.update_traces(textinfo="percent+label", marker=dict(colors=power_content["color"]))

fig.show()


## Generation Cost by Project
This shows contract costs, Nodal costs, and storage revenues

In [ ]:
gen_costs = generator_costs(
    costs_by_gen,
    storage_dispatch,
    hybrid_pair,
    gen_cap,
    generation_projects_info,
    storage_exists,
)

generator_costs_melted = gen_costs.drop(columns=["Congestion Cost", "Total Cost"]).melt(
    id_vars="generation_project", var_name="Cost", value_name="$/MWh"
)

generator_cost_fig = px.bar(
    generator_costs_melted,
    title=f"Average Generator Cost per MWh Generated ({financial_year}$)",
    x="generation_project",
    y="$/MWh",
    text="$/MWh",
    color="Cost",
    category_orders={
        "Cost": [
            "Energy Contract Cost",
            "Capacity Contract Cost",
            "Pnode Revenue",
            "Delivery Cost",
            "Congestion Cost",
            "Storage Arbitrage Revenue",
        ]
    },
    color_discrete_map={
        "Energy Contract Cost": "Red",
        "Capacity Contract Cost": "Orange",
        "Curtailed Energy Cost": "lightpink",
        "Delivery Cost": "lightblue",
        "Pnode Revenue": "blue",
        "Congestion Cost": "Pink",
        "Storage Arbitrage Revenue": "purple",
    },
).update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor="black")
generator_cost_fig.update_traces(textposition="inside")
generator_cost_fig.add_scatter(
    x=gen_costs.generation_project,
    y=gen_costs["Total Cost"],
    mode="markers+text",
    text=gen_costs["Total Cost"],
    textposition="top center",
    line=dict(color="black", width=1),
    name="Total Cost",
)

generator_cost_fig.show()



## Generator Utilization

In [ ]:
utilization = calculate_generator_utilization(dispatch)
utilization

# Costs

In [ ]:
hourly_costs = hourly_cost_of_power(
    system_power,
    costs_by_tp,
    ra_summary,
    gen_cap,
    storage_dispatch,
    fixed_costs,
    storage_exists,
)
curtailment_credit = calculate_buyer_curtailment_credit(
    costs_by_gen, generation_projects_info, gen_cap
)
cost_table = construct_cost_table(
    hourly_costs, load_balance, rec_value, financials, model_year, curtailment_credit, td_losses
)
if base_year != financial_year:
    print(
        f'Model year ({model_year}) costs are discounted to present value in {base_year} using a {financials.loc[0,"discount_rate"]*100}% discount rate'
    )
display(cost_table.set_index("Cost Component"))



# Portfolio Dispatch

In [ ]:
dispatch_by_tech, load_line, storage_charge, dispatch_fig = build_dispatch_plot(
    generation_projects_info,
    dispatch,
    storage_dispatch,
    load_balance,
    system_power,
    technology_color_map,
    storage_exists,
)
dispatch_fig.show()


## Wholesale prices

In [ ]:
build_nodal_prices_plot(
    nodal_prices, timestamps, generation_projects_info, model_year
).show()



# Energy Storage Metrics

## Battery state of charge

In [ ]:
if storage_exists:
    build_state_of_charge_plot(
        storage_dispatch, storage_builds, generation_projects_info
    ).show()
else:
    print("Storage not modeled")


## Battery Cycling and State of Charge Stats

In [ ]:
if storage_exists:
    display(
        construct_storage_stats_table(
            storage_cycle_count, storage_builds, storage_dispatch
        )
    )
else:
    print("Storage not modeled")


# Monthly positions

### Month hour average dispatch

In [ ]:
build_month_hour_dispatch_plot(
    dispatch_by_tech, load_line, storage_charge, technology_color_map, storage_exists
).show()



### Month-hour average net position

Positive values represent excess generation  
Negative values represent an open position

In [ ]:
build_open_position_plot(load_balance, storage_exists).show()


## Resource Adequacy Position

In [ ]:
if ra_exists:
    build_ra_open_position_plot(ra_summary).show()
else:
    print("Resource Adequacy was not modeled")



# Impact Metrics

## Long-run Marginal Emissions Impact
Estimates the consequential long run impact of additional generation and storage dispatch in the portfolio for the lifetime of the contract. This metric quantifies emissions impacts resulting from changes in generator dispatch, commitment, investments, and retirements.

In [ ]:
# determine which generation is additional
addl_dispatch, addl_storage_dispatch = determine_additional_dispatch(
    generation_projects_info, dispatch, storage_dispatch, storage_exists
)

# pivot the lrmer data
lrmer_data["timestamp"] = pd.to_datetime(lrmer_data["timestamp"])
lrmer_pivot = lrmer_data.pivot(
    index=["cambium_scenario", "timestamp"], columns="cambium_region", values="lrmer"
)

# multiply the generation and mer data
lr_generation_impact = lrmer_pivot.mul(addl_dispatch, axis=0, level=1).sum(axis=1)
# calculate annual total for each scenario
lr_generation_impact = (
    lr_generation_impact.reset_index().groupby("cambium_scenario").sum()
)
lr_generation_impact = lr_generation_impact.rename(
    columns={0: f'Generation {emissions_unit.split("/")[0]}'}
)

# multiply the storage and mer data
lr_storage_impact = lrmer_pivot.mul(addl_storage_dispatch, axis=0, level=1).sum(axis=1)
# calculate annual total for each scenario
lr_storage_impact = lr_storage_impact.reset_index().groupby("cambium_scenario").sum()
lr_storage_impact = lr_storage_impact.rename(
    columns={0: f'Storage {emissions_unit.split("/")[0]}'}
)

lr_impact = pd.concat([lr_generation_impact, lr_storage_impact], axis=1)
lr_impact[f'Total {emissions_unit.split("/")[0]}'] = lr_impact.sum(axis=1)

# calculate the net portfolio generation
net_generated_mwh = (
    load_balance[
        [
            "ZoneTotalGeneratorDispatch",
            "ZoneTotalStorageDischarge",
            "ZoneTotalExcessGen",
        ]
    ]
    .sum()
    .sum()
    - load_balance[["ZoneTotalStorageCharge"]].sum().sum()
)

# calculate the impact per MWh generated
lr_impact[f"Total {emissions_unit}"] = lr_impact[
    f'Total {emissions_unit.split("/")[0]}'
] / (net_generated_mwh)

# append the average values
lr_impact = pd.concat(
    [lr_impact, pd.DataFrame(lr_impact.mean(axis=0).rename("Average")).T]
)

lr_impact


## Impact on the Systemwide Daily Net Demand Peak
This metric shows how additional wind and solar generation (categorized as "future" builds in the generator portfolio chart) affect the CAISO system net demand profile, specifically how the addition of these resources increase or decrease the systemwide net demand peak. Impacts are shown both based just on wind and solar generation, and for wind, solar, and storage dispatch.

In [ ]:
addl_var_dispatch, add_storage_dispatch_2 = determine_additional_variable_dispatch(
    generation_projects_info, portfolio, dispatch, storage_dispatch, storage_exists
)

# calculate system metrics
if not addl_var_dispatch.empty:
    peaks = compare_system_peaks(
        cambium_model_year, addl_var_dispatch, add_storage_dispatch_2
    )
    display(peaks)
else:
    print("No additional generators")



## Impact on Steepness of the Systemwide Daily Maximum Net Demand Ramp
This metric shows how additional wind and solar generation (categorized as "future" builds in the generator portfolio chart) affect the CAISO system net demand profile, specifically how the addition of these resources increase or decrease the steepness of the daily maximum 3 hour ramp. This ramp typically occurs in the evening as the sun goes down, and is one of the challenges of renewable integration. Impacts are shown both based just on wind and solar generation, and for wind, solar, and storage dispatch.

In [ ]:
# calculate daily 3 hour ramp
ramp_length = 3

if not addl_dispatch.empty:
    ramps = compare_system_ramps(
        cambium_model_year, addl_var_dispatch, add_storage_dispatch_2, ramp_length
    )
    display(ramps)
else:
    print("No additional generators")



# Assumptions

In [ ]:
pd.set_option("display.max_rows", 100)
gen_assumptions = pd.read_csv(
    inputs_dir / "generation_projects_info.csv",
    usecols=[
        "GENERATION_PROJECT",
        "gen_tech",
        "gen_energy_source",
        "ppa_energy_cost",
        "ppa_capacity_cost",
        "gen_capacity_limit_mw",
    ],
)
gen_assumptions = gen_assumptions[gen_assumptions["gen_tech"] != "Storage"]
gen_assumptions = gen_assumptions.sort_values(by="GENERATION_PROJECT")
gen_assumptions = gen_assumptions.set_index("GENERATION_PROJECT")
display(gen_assumptions)


# Storage Assumptions

In [ ]:
if storage_exists:
    storage_assumptions = pd.read_csv(
        inputs_dir / "generation_projects_info.csv",
        usecols=[
            "GENERATION_PROJECT",
            "gen_tech",
            "gen_energy_source",
            "ppa_capacity_cost",
            "gen_capacity_limit_mw",
            "storage_roundtrip_efficiency",
            "storage_charge_to_discharge_ratio",
            "storage_energy_to_power_ratio",
            "storage_leakage_loss",
            "storage_hybrid_generation_project",
            "storage_hybrid_min_capacity_ratio",
            "storage_hybrid_max_capacity_ratio",
        ],
    )
    # change capacity cost to $/kw-mo
    storage_assumptions["ppa_capacity_cost"] = (
        storage_assumptions["ppa_capacity_cost"] / 12000
    )

    storage_assumptions = storage_assumptions.rename(
        columns={
            "storage_roundtrip_efficiency": "RTE",
            "storage_charge_to_discharge_ratio": "charge/discharge_ratio",
            "storage_energy_to_power_ratio": "storage_hours",
            "storage_leakage_loss": "soc_leakage_loss",
            "storage_hybrid_generation_project": "paired_hybrid_gen",
            "storage_hybrid_min_capacity_ratio": "hybrid_min_capacity_ratio",
            "storage_hybrid_max_capacity_ratio": "hybrid_max_capacity_ratio",
        }
    )

    storage_assumptions = storage_assumptions[
        storage_assumptions["gen_tech"] == "Storage"
    ]
    storage_assumptions = storage_assumptions.sort_values(by="GENERATION_PROJECT")
    storage_assumptions = storage_assumptions.set_index("GENERATION_PROJECT")
    storage_assumptions = storage_assumptions[
        [
            "gen_tech",
            "ppa_capacity_cost",
            "gen_capacity_limit_mw",
            "RTE",
            "storage_hours",
            "charge/discharge_ratio",
            "soc_leakage_loss",
            "paired_hybrid_gen",
            "hybrid_min_capacity_ratio",
            "hybrid_max_capacity_ratio",
        ]
    ]
    display(storage_assumptions)
else:
    print("Storage not modeled")


In [ ]:
# export scenario summary
# create an output summary table
construct_summary_output_table(
    scenario_name,
    cost_table,
    load_balance,
    portfolio,
    sensitivity_table,
    lr_impact,
    total_emissions,
    peaks,
    ramps,
    emissions_unit,
    base_year,
    financial_year,
    dispatch,
).to_csv(scenario_output_dir / f"scenario_summary_{scenario_name}.csv")

